In [7]:
from stratopy.extractors import cloudsat
from stratopy.extractors import goes
from stratopy.transformers import mergers
from stratopy.transformers import scalers

Download 1 band file from ABI GOES16

In [4]:
goes_obj = goes.GOES16(product_type="ABI-L2-CMIPF", channel=13)
goes_data = goes_obj.fetch("2019 dec 12 15:00")

Lets see one transformer. This one normalizes an ABI image to [0,1].

Returns a numpy array of shape (n_bands, image_width, image_height).

In [9]:
norm_goes = scalers.MinMaxNormalize().transformer(goes_data)
norm_goes[:,2500,2500]

array([0.28565142], dtype=float32)

Download a file from CloudSat.

In [ ]:
csat_obj = cloudsat.CloudSat(product_type="", username="", keyfile=, keypass="")
csat_data = csat_obj.fetch("2019 dec 12 15:00")

Merge products

In [ ]:
merged_data = mergers.MergePolarGeos("2019 dec 12 15:00", 
                                     time_zone="UTC", 
                                     trim_size=(3,3), 
                                     norm=True).transformer(csat_data, goes_data)